## Loading images from directory as dataset

In [1]:
img_width, img_height = 100, 100
input_shape = (img_width, img_height, 1)
batch_size = 32
output_classes = 62

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1/255,
                                     shear_range=0.2, zoom_range=0.2)
data_generator = ImageDataGenerator(rescale=1/255)
test_generator = ImageDataGenerator(rescale=1/255)

train_ds = train_generator.flow_from_directory(
    directory="./train",
    target_size = (img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

validation_ds = data_generator.flow_from_directory(
    directory="./val",
    target_size = (img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

test_ds = test_generator.flow_from_directory(
    directory="./test",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

Found 2356 images belonging to 62 classes.
Found 682 images belonging to 62 classes.
Found 372 images belonging to 62 classes.


## Building CNN model

### Importing required packages

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation
from keras.layers import Dropout, BatchNormalization

### Define the VGGNet model

In [6]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_classes, activation='softmax'))

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 100, 100, 64)      640       
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 50, 50, 128)       73856     
                                                                 
 conv2d_5 (Conv2D)           (None, 50, 50, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 128)      0         
 2D)                                                  

In [8]:
from keras import optimizers

model.compile(
    optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

## Train/Fit The model

In [9]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=6,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

In [10]:
epochs_range = 100
# Train the model
history = model.fit(train_ds, batch_size=32, 
                    epochs=epochs_range,callbacks=None,
                    validation_data=validation_ds)

Epoch 1/100
 9/74 [==>...........................] - ETA: 8:51 - loss: 4.1898 - accuracy: 0.0104

KeyboardInterrupt: 

##Plot the result

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(range(len(acc)), acc, label='Training Accuracy')
plt.plot(range(len(val_acc)), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(range(len(loss)), loss, label='Training Loss')
plt.plot(range(len(val_loss)), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

##Save the model

In [ ]:
# Save the model as model.h5
model.save('model_transfer.h5')

In [ ]:
# Print the class indices 
print("Prediction Dict: ", train_ds.class_indices)